In [24]:
from termcolor import colored
import os 
from PyPDF2 import PdfReader
from langchain.llms import OpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain.callbacks import get_openai_callback
from termcolor import colored
os.environ["OPENAI_API_KEY"] = #INSERT YOUR KEY HERE#
def read_PDF(pdf_dir):
    doc_reader = PdfReader(pdf_dir)
    raw_text = ''
    for i, page in enumerate(doc_reader.pages):
        text = page.extract_text()
        if text:
            raw_text += text
    return raw_text

# Splits raw text into smaller chunks with overlap
def split_text(raw_text):
    text_splitter = CharacterTextSplitter(        
        separator = "\n",
        chunk_size = 2500,
        chunk_overlap  = 400, #striding over the text
        length_function = len,
    )
    texts = text_splitter.split_text(raw_text)
    return texts

directory = r"C:\Users\felip\OneDrive - Fundacao Getulio Vargas - FGV\!_Paper04_Chat_PDF\TCE_PIAUI\cross_reference"
files = os.listdir(directory)
pdfs = [file for file in files if file.lower().endswith('.pdf')]
print(len(pdfs),' PDFs were found!\n\n')

3  PDFs were found!




In [25]:
full_text = ''
for k in pdfs:
    full_text = full_text + read_PDF(directory+'\\'+k)

In [26]:
len(full_text)

32681

In [36]:
prompt_CR = """
Your Role: An auditor expert in extracting information from multiple sources. 
Your Task: Verify informations dispersed in 2 documents that forms a full context.
Using only information from these context: {context}
Question: {question}
Do not add external information on the replay and do not include opinions.
The answer is:
"""
question = """What was the external auditor's voting recommendation was for the municipality's accounts?
What was the rapporteur voting recommendation?"""
question = "Did the rapporteur use information provided by the external audit work to issue voting recommendations?"
question = "Was the vote in the Prior Opinion (Parecer Prévio) the same or different from the rapporteur's voting recommendation?"

prompt_cross_ref = PromptTemplate(template=prompt_CR, input_variables=['context', 'question'])

def cost_and_tokens(usage_list):  
    total_cost = 0
    total_tokens = 0
    for c in usage_list:
        total_cost += c.total_cost
        total_tokens += c.total_tokens
    return total_cost,total_tokens

def QA_LLM(query,prompt_input):
    
    QA = RetrievalQA.from_chain_type(llm=OpenAI(model_name='gpt-3.5-turbo',#cost of $0.0015 / 1K tokens
                                                temperature = 0), 
                                      chain_type="stuff", 
                                      retriever=retriever,
                                      #verbose=True,
                                      chain_type_kwargs={"prompt": prompt_input},
                                      return_source_documents=True)
    with get_openai_callback() as cost:
        answer = QA(query,prompt_input)     
    return answer, cost

In [37]:
embeddings = OpenAIEmbeddings()
texts = split_text(full_text)
docsearch = FAISS.from_texts(texts, embeddings)
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":4})
pattern, pattern_cost = QA_LLM(question,prompt_cross_ref) 

In [52]:
question = "A recomendação de voto emitida no Prior Opnion (Parecer Prévio) foi igual a recomendação de voto do relator? Qual foi a recomendação?"
question = "Was the voting recommendation issued in the Prior Opnion the same as the rapporteur's voting recommendation? What was the recommendation?"
question = "Did the rapporteur use information provided by the external audit work to issue voting recommendations?Was the voting recommendation issued in the Prior Opnion the same as the rapporteur's voting recommendation? What was the recommendation?"

pattern, pattern_cost = QA_LLM(question,prompt_cross_ref) 

In [53]:
pattern['result']

"Based on the information provided in the documents, it is not explicitly stated whether the rapporteur used information provided by the external audit work to issue voting recommendations. However, it is mentioned that the Director of Municipal Administration Audit Department (DFAM) of the Tribunal of Accounts produced a preliminary report, which identified several occurrences in the municipality's annual accounts. The mayor was given the opportunity to provide justifications and defense but did not do so.\n\nRegarding the voting recommendation, it is stated that the rapporteur issued a prior opinion recommending the Approval with Reservations of the government accounts of the Municipal Executive Head of Acauã-PI for the financial year 2017. The specific details of the recommendation are not provided in the given information."

In [14]:
pattern['source_documents']

[Document(page_content='Do confronto entre o relatório produzido pela Diretoria de Fiscalização da Administração \nMunicipal e diante da ausência de justificativa e defesa, reiteram -se as ocorrências constatadas \nno relatório preliminar : \n \n \n \n \n  \n  Estado do Piauí  \nTribunal de Contas  Processo 006867/2018  \nPendente de  \njulgamento  \n \n  \n    \n2 2.1 INGRESSO EXTEMPORÂNEO DA PRESTAÇÃO DE CONTAS MENSAL : O Prefeito \nMunicipal, responsável pelo envio da prestação de contas mensal, apresentou ao Tribunal  de \nContas, a referida documentação, nos prazos indicados:  \nPeríodo  Prazo Legal  Sagres -Contábil  Sagres Folha  Média Atraso  Entrega  Atraso  Entrega  Atraso  \nJaneiro  02/05/2017  27/03/2018  329 15/03/2017  0 164 \nFevereiro  15/05/2017  27/03/2018  316 15/03/2017  0 158 \nMarço  02/06/2017  28/03/2018  299 16/05/2017  0 149 \nAbril 03/07/2017  28/03/2018  268 25/05/2017  0 134 \nMaio 31/07/2017  28/03/2018  240 18/07/2017  0 120 \nJunho  01/09/2017  28/03/20